In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_reparam_space_time as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

In [7]:
a,b = np.exp(1.8060), np.exp(0.7948)

range_lon_inv = 1/a
range_lat_inv = 1/(np.sqrt(b)*a) 

print(range_lat_inv, range_lon_inv)

0.11042706734093927 0.16431006433040046


Load monthly data

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[0,5]      
lon_range_input=[123, 133.0] 

df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input

)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0, 5], lon: [123, 133.0]


Load daily data applying max-min ordering

In [4]:
daily_aggregated_tensors_dw = [] 
daily_hourly_maps_dw = []      

daily_aggregated_tensors_vecc = [] 
daily_hourly_maps_vecc = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_dw.append( day_aggregated_tensor )
    daily_hourly_maps_dw.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_vecc.append( day_hourly_map )
print(daily_aggregated_tensors_vecc[0].shape)
#print(daily_hourly_maps[0])
nn = daily_aggregated_tensors_vecc[0].shape[0]

torch.Size([145008, 4])


Hyper parameters

In [7]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 400
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

Admas full batch

In [ ]:
import torch
import numpy as np
import time

# --- Assume global variables are set: ---
# daily_hourly_maps, daily_aggregated_tensors, nns_map
# lat_lon_resolution, v, mm_cond_number, nheads
# lr, patience, factor, epochs

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] # 0 index 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors_vecc[day_idx]

    # --- 💥 Correct Parameter Initialization (SPATIO-TEMPORAL) ---
    
    init_sigmasq   = 13.0031
    init_range_lat = 0.1533
    init_range_lon = 0.1942
    init_nugget    = 0.2312 
    init_range_time  = 1.2751  # <-- NEW (Temporal range)
    init_advec_lat = 0.0254  # <-- NEW (Start with no advection)
    init_advec_lon = -0.1641  # <-- NEW (Start with no advection)
    '''  
    init_sigmasq   = 11.7969
    init_range_lat = 0.1104
    init_range_lon = 0.1643
    init_nugget    = 0.0001 
    init_range_time  = 1.28  # <-- NEW (Temporal range)
    init_advec_lat = 0.0223  # <-- NEW (Start with no advection)
    init_advec_lon = -0.1672  # <-- NEW (Start with no advection)
    '''

    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / beta)^2
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 💥 7-parameter spatio-temporal list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log_phi1
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log_phi2
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log_phi3
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log_phi4
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (NOT log)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (NOT log)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log_nugget
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 Group all 7 parameters.
    # For a more advanced setup, you could create a separate group
    # for advection params [4, 5] with a different (e.g., smaller) LR.
    lr_all = lr 
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the Correct Class ---
    # Assumes fit_vecchia_adams is defined in your current scope
    model_instance = kernels_reparam_space_time.fit_vecchia_adams(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- Set Optimizer (This part is unchanged) ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        
            factor=factor             
        )

    # --- 💥 Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_model(  # <-- Changed method name
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_aniso_STABLE_log_reparam, # <-- Changed covariance function
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")

    # 30m per 10. #epoch 120: 210 m. 331m


--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2041
  Param 1: 1.6389
  Param 2: 0.4730
  Param 3: -3.7638
  Param 4: 0.0254
  Param 5: -0.1641
  Param 6: -1.4645


/opt/anaconda3/envs/faiss_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


--- Epoch 1 / Loss: 1.242094 ---
  Param 0: Value=4.2041, Grad=-0.0019605295156889624
  Param 1: Value=1.6389, Grad=0.00016578177948395437
  Param 2: Value=0.4730, Grad=-0.00047301335143176324
  Param 3: Value=-3.7638, Grad=-7.368123993227674e-05
  Param 4: Value=0.0254, Grad=0.0015114695661382896
  Param 5: Value=-0.1641, Grad=0.001130199229753584
  Param 6: Value=-1.4645, Grad=0.0001448594927842602
  Max Abs Grad: 1.960530e-03
------------------------------
--- Epoch 11 / Loss: 1.242767 ---
  Param 0: Value=4.2558, Grad=0.020533918042005113
  Param 1: Value=1.6332, Grad=-0.002348116442590908
  Param 2: Value=0.5138, Grad=0.006276503570925667
  Param 3: Value=-3.5709, Grad=0.0006742105277630215
  Param 4: Value=-0.0048, Grad=-0.010196574750007526
  Param 5: Value=-0.1739, Grad=-0.0033543686534067605
  Param 6: Value=-1.6550, Grad=0.001350780128044063
  Max Abs Grad: 2.053392e-02
------------------------------
--- Epoch 21 / Loss: 1.242143 ---
  Param 0: Value=4.2116, Grad=-0.008534688

--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2739
  Param 1: 1.8061
  Param 2: 0.7952
  Param 3: -4.1058
  Param 4: 0.0223
  Param 5: -0.1672
  Param 6: -9.2103
/opt/anaconda3/envs/faiss_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
--- Epoch 1 / Loss: 1.245499 ---
  Param 0: Value=4.2739, Grad=0.017745675058296936
  Param 1: Value=1.8061, Grad=0.0008265844799274488
  Param 2: Value=0.7952, Grad=0.021193197189964156
  Param 3: Value=-4.1058, Grad=-0.0016905664152934525
  Param 4: Value=0.0223, Grad=0.001970792580072817
  Param 5: Value=-0.1672, Grad=0.012366998773789932
  Param 6: Value=-9.2103, Grad=6.652335220761932e-07
  Max Abs Grad: 2.119320e-02
------------------------------
--- Epoch 11 / Loss: 1.243057 ---
  Param 0: Value=4.3370, Grad=0.005532713651162728
  Param 1: Value=1.8698, Grad=0.0015965380205475465
  Param 2: Value=0.3906, Grad=-0.0031288813856308797
  Param 3: Value=-3.5079, Grad=0.00012640251644008914
  Param 4: Value=-0.0323, Grad=-0.019579353112110447
  Param 5: Value=-0.1245, Grad=0.005058925449702462
  Param 6: Value=-9.2022, Grad=2.4863423479562597e-07
  Max Abs Grad: 1.957935e-02
------------------------------

--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 3.0647
  Param 1: 0.3567
  Param 2: 0.1177
  Param 3: -4.6052
  Param 4: 0.0200
  Param 5: -0.0800
  Param 6: 0.4055

  Max Abs Grad: 3.261639e-01
------------------------------
--- Epoch 11 / Loss: 1.264441 ---
  Param 0: Value=3.6519, Grad=0.033102462085258695
  Param 1: Value=0.0235, Grad=-0.0007389134000692495
  Param 2: Value=0.6861, Grad=0.012528458699466947
  Param 3: Value=-4.0819, Grad=-0.0006021767918773808
  Param 4: Value=0.0095, Grad=-0.0008643903938066648
  Param 5: Value=-0.4955, Grad=-0.011065596143078432
  Param 6: Value=0.7710, Grad=0.048430986604446245
  Max Abs Grad: 4.843099e-02
------------------------------
--- Epoch 21 / Loss: 1.254908 ---
  Param 0: Value=3.5690, Grad=-0.017414686875716533
  Param 1: Value=0.3038, Grad=-0.00031863693271167153
  Param 2: Value=0.4878, Grad=-0.0031871705493084403
  Param 3: Value=-3.9547, Grad=-0.0009551805522446299
  Param 4: Value=-0.0349, Grad=-0.003536394381183513
  Param 5: Value=-0.5049, Grad=-0.009332212642619617
  Param 6: Value=0.4779, Grad=0.0035137438635113425
  Max Abs Grad: 1.741469e-02
------------------------------
--- Epoch 31 / Loss: 1.251845 ---
  Param 0: Value=3.6745, Grad=-0.01555689658857398
  Param 1: Value=0.5720, Grad=-0.0005128375911674786
  Param 2: Value=0.5004, Grad=-0.002206009817795905
  Param 3: Value=-3.8553, Grad=-0.0007826904517286726
  Param 4: Value=0.0247, Grad=-0.00032502390531596485
  Param 5: Value=-0.4514, Grad=-0.011680323865716345
  Param 6: Value=0.3269, Grad=0.002009819891799335
  Max Abs Grad: 1.555690e-02
------------------------------
--- Epoch 41 / Loss: 1.248747 ---
  Param 0: Value=3.8277, Grad=0.006262703404085881
  Param 1: Value=0.9707, Grad=-0.0011425011020983831
  Param 2: Value=0.5399, Grad=0.004287867156494423
  Param 3: Value=-3.7662, Grad=-0.00028585190495268624
  Param 4: Value=0.0469, Grad=0.0025113508266898796
  Param 5: Value=-0.3520, Grad=-0.014885347933420185
  Param 6: Value=0.2267, Grad=0.011262560746062734
  Max Abs Grad: 1.488535e-02
------------------------------
--- Epoch 51 / Loss: 1.245564 ---
  Param 0: Value=3.8707, Grad=-0.009092823529823066
  Param 1: Value=1.3319, Grad=0.0015285008232184647
  Param 2: Value=0.4458, Grad=-0.0017986285273207225
  Param 3: Value=-3.7123, Grad=5.181327027115963e-05
  Param 4: Value=0.0220, Grad=-0.0026806427888267865
  Param 5: Value=-0.2095, Grad=-0.0054568132706633
  Param 6: Value=0.0793, Grad=0.0038554147044682343
  Max Abs Grad: 9.092824e-03
------------------------------
--- Epoch 61 / Loss: 1.244809 ---
  Param 0: Value=3.9545, Grad=0.0025311473891214244
  Param 1: Value=1.1901, Grad=-0.0009350653303357313
  Param 2: Value=0.4366, Grad=0.0010677078901593694
  Param 3: Value=-3.6776, Grad=-0.00048638118522927384
  Param 4: Value=0.0279, Grad=-0.000609149651126906
  Param 5: Value=-0.1399, Grad=0.012523378927349654
  Param 6: Value=-0.0443, Grad=0.006097463420966644
  Max Abs Grad: 1.252338e-02

# L bfgs

In [8]:
import torch
import numpy as np
import torch.optim as optim
import time
# NOTE: Assuming the necessary classes and global variables are defined elsewhere, 
# specifically the referenced 'fit_vecchia_lbfgs' class and 'kernels_reparam_space_time' module.

# --- Assume global variables are set: ---
# daily_hourly_maps, daily_aggregated_tensors, nns_map
# lat_lon_resolution, v, mm_cond_number, nheads
# lr, patience, factor, epochs

# --- L-BFGS SPECIFIC GLOBAL PARAMETERS ---
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 20       # Number of outer optimization steps 10 to 20
LBFGS_HISTORY_SIZE = 100   # Memory for Hessian approximation
LBFGS_MAX_EVAL = 80        # Max evaluations (line search) per step 50 to 80

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] # 0 index 
for day_idx in day_indices:  

    # Assuming data access is correct
    daily_hourly_map = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors_vecc[day_idx]

    # --- Parameter Initialization (SPATIO-TEMPORAL) ---
    '''  
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_range_time = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    '''
    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.135
    init_range_time = 0.7
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689

    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 7-parameter spatio-temporal list (Log/Linear)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log(phi4)
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (linear)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (linear)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log(nugget)
    ]

    # --- Define parameter groups ---
    lr_all = LBFGS_LR
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    # L-BFGS requires the parameters to be iterable in a single list or group
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info (using placeholder print variables) ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the L-BFGS Class ---
    # NOTE: Assuming fit_vecchia_lbfgs is available via kernels_reparam_space_time
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 Set L-BFGS Optimizer ---
    # L-BFGS specific arguments are passed here
    optimizer = model_instance.set_optimizer(
            param_groups,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        # max_iter in LBFGS is the line search limit
            history_size=LBFGS_HISTORY_SIZE,
            tolerance_grad = 1e-7,
            tolerance_change = 1e-12
        )

    # --- 💥 Call the L-BFGS Fit Method ---
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_aniso_STABLE_log_reparam, 
            max_steps=LBFGS_MAX_STEPS, # Outer loop steps
            grad_tol = 1e-7
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} L-BFGS steps.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")

    # 85 min 1 step 96m steo 2 finished!


--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -2.5562
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -2.0025
--- Starting L-BFGS Optimization ---
--- Step 1/20 / Loss: 1.243818 ---
  Param 0: Value=4.2824, Grad=3.7316651559889873e-07
  Param 1: Value=1.7108, Grad=-4.9941013494580584e-08
  Param 2: Value=0.4883, Grad=2.2757637405767504e-07
  Param 3: Value=-3.7686, Grad=7.4054935999273844e-09
  Param 4: Value=0.0201, Grad=-1.2531451323882498e-07
  Param 5: Value=-0.1611, Grad=-8.692719141656638e-08
  Param 6: Value=-5.1617, Grad=7.410251173775746e-10
  Max Abs Grad: 3.731665e-07
------------------------------
--- Step 2/20 / Loss: 1.241995 ---
  Param 0: Value=4.2824, Grad=3.7316651559889873e-07
  Param 1: Value=1.7108, Grad=-4.9941013494580584e-08
  Param 2: Value=0.4883, Grad=2.2757637405767504e-07
  Param 3: Value=-3.7686, Grad=7.4054935999273844e-09


KeyboardInterrupt: 

In [13]:
60*117

7020

--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -3.7677
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
--- Starting L-BFGS Optimization ---
--- Step 1/10 / Loss: 1.242106 ---
  Param 0: Value=4.2866, Grad=6.404439975316003e-07
  Param 1: Value=1.7396, Grad=-5.182434301931914e-07
  Param 2: Value=0.4891, Grad=6.42059345455452e-07
  Param 3: Value=-3.7771, Grad=-1.810516477001804e-08
  Param 4: Value=0.0205, Grad=1.0994677672789547e-06
  Param 5: Value=-0.1641, Grad=1.23514462006375e-06
  Param 6: Value=-11.8133, Grad=1.01352131413913e-09
  Max Abs Grad: 1.235145e-06
------------------------------

Converged on gradient norm (max|grad| < 1e-05) at step 1
FINAL STATE: Step 1, Loss: 1.2421059448955856
  Raw (vecc) Parameters: [4.286642426458099, 1.7396166546213205, 0.48908102342543236, -3.777078889337886, 0.020486826416164613, -0.16410986090084237, -11.813252497216158]
  Interpretable Parameters:
    sigma_sq  : 12.769069
    range_lon : 0.175588
    range_lat : 0.137496
    beta      : 0.151293
    advec_lat : 0.020487
    advec_lon : -0.164110
    nugget    : 0.000007
Day 1 optimization finished in 5777.95s over 1 L-BFGS steps.
Day 1 final results (raw params + loss): [4.286642426458099, 1.7396166546213205, 0.48908102342543236, -3.777078889337886, 0.020486826416164613, -0.16410986090084237, -11.813252497216158, 1.2421059448955856]

In [ ]:
--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -3.7677
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
--- Starting L-BFGS Optimization ---
--- Step 1/10 / Loss: 1.242106 ---
  Param 0: Value=4.2866, Grad=6.404439975316003e-07
  Param 1: Value=1.7396, Grad=-5.182434301931914e-07
  Param 2: Value=0.4891, Grad=6.42059345455452e-07
  Param 3: Value=-3.7771, Grad=-1.810516477001804e-08
  Param 4: Value=0.0205, Grad=1.0994677672789547e-06
  Param 5: Value=-0.1641, Grad=1.23514462006375e-06
  Param 6: Value=-11.8133, Grad=1.01352131413913e-09
  Max Abs Grad: 1.235145e-06
------------------------------

Converged on gradient norm (max|grad| < 1e-05) at step 1
FINAL STATE: Step 1, Loss: 1.2421059448955856
  Raw (vecc) Parameters: [4.286642426458099, 1.7396166546213205, 0.48908102342543236, -3.777078889337886, 0.020486826416164613, -0.16410986090084237, -11.813252497216158]
  Interpretable Parameters:
    sigma_sq  : 12.769069
    range_lon : 0.175588
    range_lat : 0.137496
    beta      : 0.151293
    advec_lat : 0.020487
    advec_lon : -0.164110
    nugget    : 0.000007
Day 1 optimization finished in 5777.95s over 1 L-BFGS steps.
Day 1 final results (raw params + loss): [4.286642426458099, 1.7396166546213205, 0.48908102342543236, -3.777078889337886, 0.020486826416164613, -0.16410986090084237, -11.813252497216158, 1.2421059448955856]